# <center><font color='blue'>SENTIMENT ANALYSIS: COVID</center></font>

## Table of contents
- [1 - Objetivos](#1)
- [2 - Librerías necesarias](#2)
- [3 - Carga y visualización de datos](#3)
- [4 - Pre-procesamiento de datos](#4)
    - [4.1. - Datos faltantes](#4.1)
    - [4.2. - Data Categóricos](#4.2)
    - [4.3. - Balanceo de clases](#4.3)
    - [4.4. - Pre-Procesamiento especial para NLP](#4.4)
- [5 - Modelos](#5)
    - [5.1. - Modelo 1](#5.1)
    - [5.2. - Modelo 2](#5.2)
    - [5.3. - Modelo 3](#5.3)
    - [5.4. - Modelo 4](#5.4)
    - [5.4. - Modelo 5](#5.5)
    - [5.4. - Modelo 6](#5.6)
    - [5.4. - Modelo 7](#5.7)
- [6 - Comparando los modelos y eligiendo el mejor](#6)
- [7 - Ajuste de hiperparámetros](#7)
- [8 - Predicciones con el modelo final](#8)
- [9 - Guardando el modelo](#9)
- [10 - Conclusiones](#10)

<a name="1"></a>
## <b> <font color='blue'> 1. Objectives </font> </b>

Practice with a natural language processing problem.
<br>
Here, given a set of tweets, analyze whether the sentiment is positive, negative, or neutral.

<a name="2"></a>
## <b> <font color='blue'> 2. Setup </font> </b>

In [1]:
# do not print info and warning messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [2]:
# basic imports
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
import pandas as pd

2024-07-11 17:53:16.979024: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 17:53:16.979061: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 17:53:16.980295: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


<a name="3"></a>
## <b> <font color='blue'> 3. Data loading and visualization </font> </b>

In [3]:
!ls data

Corona_NLP_test.csv  Corona_NLP_train.csv


We can see that we have 2 datasets, one for training and one for test.

In [4]:
train_data = pd.read_csv('data/Corona_NLP_train.csv',encoding='latin-1')
test_data = pd.read_csv('data/Corona_NLP_test.csv',encoding='latin-1')

In [5]:
# let's see some of the training data
pd.set_option('display.max_colwidth', None) # que No recorte el texto

train_data.head(10)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order,Positive
2,3801,48753,Vagabonds,16-03-2020,"Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P",Positive
3,3802,48754,NaN,16-03-2020,"My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\r\r\n#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://t.co/zrlG0Z520j",Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don't panic. It causes shortage...\r\r\n\r\r\n#CoronavirusFrance #restezchezvous #StayAtHome #confinement https://t.co/usmuaLq72n",Extremely Negative
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,"As news of the regionÂs first confirmed COVID-19 case came out of Sullivan County last week, people flocked to area stores to purchase cleaning supplies, hand sanitizer, food, toilet paper and other goods, @Tim_Dodson reports https://t.co/cfXch7a2lU",Positive
6,3805,48757,"35.926541,-78.753267",16-03-2020,"Cashier at grocery store was sharing his insights on #Covid_19 To prove his credibility he commented ""I'm in Civics class so I know what I'm talking about"". https://t.co/ieFDNeHgDO",Positive
7,3806,48758,Austria,16-03-2020,Was at the supermarket today. Didn't buy toilet paper. #Rebel\r\r\n\r\r\n#toiletpapercrisis #covid_19 https://t.co/eVXkQLIdAZ,Neutral
8,3807,48759,"Atlanta, GA USA",16-03-2020,"Due to COVID-19 our retail store and classroom in Atlanta will not be open for walk-in business or classes for the next two weeks, beginning Monday, March 16. We will continue to process online and phone orders as normal! Thank you for your understanding! https://t.co/kw91zJ5O5i",Positive
9,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,"For corona prevention,we should stop to buy things with the cash and should use online payment methods because corona can spread through the notes. Also we should prefer online shopping from our home. It's time to fight against COVID 19?. #govindia #IndiaFightsCorona",Negative


In [6]:
print(f" Training data: (rows, cols) = {train_data.shape}")
print(f" Training data type:  {type(train_data)}")
print(f" Test data: (rows, cols) = {test_data.shape}")
print(f" Test data type:  {type(test_data)}")

 Training data: (rows, cols) = (41157, 6)
 Training data type:  <class 'pandas.core.frame.DataFrame'>
 Test data: (rows, cols) = (3798, 6)
 Test data type:  <class 'pandas.core.frame.DataFrame'>


<a name="4"></a>
## <b> <font color='blue'> 4. Data Pre-processing </font> </b>


<a name="4.1"></a>
### <b> <font color='#5499C7'> 4.1. Handling missing data </font> </b>

In [7]:
print(f'Missing data train:\n {train_data.isnull().sum()} \n')
print(f'Missing data test:\n {test_data.isnull().sum()}')

Missing data train:
 UserName            0
ScreenName          0
Location         8590
TweetAt             0
OriginalTweet       0
Sentiment           0
dtype: int64 

Missing data test:
 UserName           0
ScreenName         0
Location         834
TweetAt            0
OriginalTweet      0
Sentiment          0
dtype: int64


There are no missing data in the columns we care about (Original Tweet and Sentiment).

<a name="4.2"></a>
### <b> <font color='#5499C7'> 4.2. Separate features from target </font> </b>

In [8]:
X_train = train_data['OriginalTweet']
y_train = train_data['Sentiment']
X_test = test_data['OriginalTweet']
y_test = test_data['Sentiment']

In [9]:
type(X_train), type(y_train)

(pandas.core.series.Series, pandas.core.series.Series)

<a name="4.3"></a>
### <b> <font color='#5499C7'> 4.3. Categorical data </font> </b>

Our target (Sentiment) is categorical, lets see the different options:

In [10]:
y_train.unique()

array(['Neutral', 'Positive', 'Extremely Negative', 'Negative',
       'Extremely Positive'], dtype=object)

Lets see if there are other options in the test set:

In [11]:
y_test.unique()

array(['Extremely Negative', 'Positive', 'Extremely Positive', 'Negative',
       'Neutral'], dtype=object)

In [12]:
are_same = np.array_equal(np.sort(y_train.unique()), np.sort(y_test.unique()))
print(f"Are unique values the same? {are_same}")

Are unique values the same? True


We are not so interested in whether a sentiment is positive or extremely positive, therefore we will rename the label extremely positive to positive.

In [13]:
y_train = y_train.replace('Extremely Positive', 'Positive').replace('Extremely Negative', 'Negative')
y_test = y_test.replace('Extremely Positive', 'Positive').replace('Extremely Negative', 'Negative')

Check

In [14]:
print(np.sort(y_train.unique()), np.sort(y_test.unique()))

['Negative' 'Neutral' 'Positive'] ['Negative' 'Neutral' 'Positive']


We will not consider that there is an ordinal relationship, therefore we will encode the labels using one-hot encoding.

Remember that we need to apply the same encoding to train and test.

In [15]:
from sklearn.preprocessing import OneHotEncoder


# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# Fit the encoder on the train set
encoder.fit(y_train.values.reshape(-1, 1))

# Transform both train and test sets
one_hot_train = encoder.transform(y_train.values.reshape(-1, 1))
one_hot_test = encoder.transform(y_test.values.reshape(-1, 1))

# Convert to DataFrame for easier inspection
y_train = pd.DataFrame(one_hot_train, columns = encoder.get_feature_names_out(['sentiment']))
y_test = pd.DataFrame(one_hot_test, columns = encoder.get_feature_names_out(['sentiment']))

print("One-hot encoded train set:")
print(y_train)
print("\nOne-hot encoded test set:")
print(y_test)

One-hot encoded train set:
       sentiment_Negative  sentiment_Neutral  sentiment_Positive
0                     0.0                1.0                 0.0
1                     0.0                0.0                 1.0
2                     0.0                0.0                 1.0
3                     0.0                0.0                 1.0
4                     1.0                0.0                 0.0
...                   ...                ...                 ...
41152                 0.0                1.0                 0.0
41153                 1.0                0.0                 0.0
41154                 0.0                0.0                 1.0
41155                 0.0                1.0                 0.0
41156                 1.0                0.0                 0.0

[41157 rows x 3 columns]

One-hot encoded test set:
      sentiment_Negative  sentiment_Neutral  sentiment_Positive
0                    1.0                0.0                 0.0
1           

In this example, handle_unknown='ignore' is used to ensure that any categories present in the test set but not in the train set are handled gracefully. This is useful when you have categories in the test set that were not seen during training.

<a name="4.4"></a>
### <b> <font color='#5499C7'> 4.4. Train/val split </font> </b>

We are going to create a validation dataset.

In [16]:
from sklearn.model_selection import train_test_split 

# Perform the split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


<a name="4.5"></a>
### <b> <font color='#5499C7'> 4.5. Class balance </font> </b>

In [17]:
y_train.value_counts(normalize=True)

sentiment_Negative  sentiment_Neutral  sentiment_Positive
0.0                 0.0                1.0                   0.438238
1.0                 0.0                0.0                   0.374670
0.0                 1.0                0.0                   0.187092
dtype: float64

We have some imbalance with respect to the neutral class. We are going to apply SMOTE to take care of this.

In [18]:
#!pip install imbalanced-learn

In [19]:
"""
from imblearn.over_sampling import SMOTE

# Applyt SMOTE only to the training set
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Show the new distribution
print(y_train_resampled.value_counts())

para smote debo tener las etiquetas en formato categórico

"""

'\nfrom imblearn.over_sampling import SMOTE\n\n# Applyt SMOTE only to the training set\nsmote = SMOTE(random_state=42)\nX_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)\n\n# Show the new distribution\nprint(y_train_resampled.value_counts())\n\npara smote debo tener las etiquetas en formato categórico\n\n'

<a name="4.6"></a>
### <b> <font color='#5499C7'> 4.6. Pre-process for NLP </font> </b>

Let's preprocess the text from OriginalTweet, for which:

- We will remove stop words.
- We will remove certain special characters, like "@".
- We will apply lemmatization.


<b>Note:</b> We will also remove punctuation, convert everything to lowercase, and tokenize later using TextVectorization.


In [20]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

import re

[nltk_data] Downloading package stopwords to /home/marcos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/marcos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/marcos/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


We will remove the stop words, some special characters and apply lemmatization:

In [21]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Inicializar lematizador
lemmatizer = WordNetLemmatizer()

In [22]:
def preprocess_text(text):
    
    # stop words
    stop_words = set(stopwords.words('english'))
    
    # Remove specific characters using regex
    text = re.sub(r'[@#]', '', text)
    
    # Remove hyperlinks
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    
    # Tokenize and lemmatize
    words = word_tokenize(text)
    filtered_words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words]
        
    return ' '.join(filtered_words)

Let's try our function:

In [23]:
text = "Hello @world! This is# a sample text."
processed_text = preprocess_text(text)
print(processed_text)

hello world ! sample text .


Let's apply ou pre-process function:

In [24]:
X_train_prep = X_train.apply(preprocess_text)
X_val_prep = X_val.apply(preprocess_text)
X_test_prep = X_test.apply(preprocess_text)

Let' see a few examples:

In [25]:
X_train[0]

'@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8'

In [26]:
X_train_prep[0]

'menyrbie phil_gahan chrisitv'

In [27]:
X_train.index

Int64Index([ 8191,  3725, 22759, 15010, 39142, 30040, 30389, 18537,   919,
            33677,
            ...
            41090, 16023, 21962, 37194, 16850,  6265, 11284, 38158,   860,
            15795],
           dtype='int64', length=32925)

In [28]:
X_train[8191]

'Unemployment claims made online in Virginia this week:\r\r\n\r\r\nMonday: 426\r\r\nTuesday: 2,150\r\r\n\r\r\nAnd the numbers are going to get bigger. https://t.co/fUeg2RL2dl'

In [29]:
X_train_prep[8191]

'unemployment claim made online virginia week : monday : 426 tuesday : 2,150 number going get bigger .'

### Text Vectorization layer

Now we will create and apply a layer called <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization" target='_blank'>TextVectorization</a>, which will remove punctuation, convert everything to lowercase, and tokenize.

In [30]:
VOCAB_SIZE = 10000
max_length = 45 # max length our sequences will be (e.g. how many words from a Tweet does a model see?)


vectorizer = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE,
                                    output_mode="int")
                                    #output_sequence_length=max_length)

# Fit the text vectorizer instance to the training data using the adapt() method
vectorizer.adapt(X_train_prep)

Let' see the 20 firsts tokens:

In [35]:
vocab = np.array(vectorizer.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'coronavirus', 'covid19', 'price', 'store',
       'supermarket', 'food', 'grocery', 'people', 'amp', 'consumer',
       '19', 'covid', 'shopping', 's', 'online', 'need', 'time',
       'pandemic'], dtype='<U29')

We are going to apply this layer to our datasets.

In [31]:
# Esta capa la aplicaremos luego en el modelo (al inicio) -> versión original


# apply to the datasets
X_train_vectors = vectorizer(X_train_prep)
X_val_vectors = vectorizer(X_val_prep)
X_test_vectors = vectorizer(X_test_prep)

In [32]:
X_train_vectors

<tf.Tensor: shape=(32925, 46), dtype=int64, numpy=
array([[ 822,  744,  232, ...,    0,    0,    0],
       [ 281,  190,    3, ...,    0,    0,    0],
       [   1, 2114, 5896, ...,    0,    0,    0],
       ...,
       [9959,    1, 2203, ...,    0,    0,    0],
       [1026,  124, 1416, ...,    0,    0,    0],
       [1344, 8453,    2, ...,    0,    0,    0]])>

<a name="4.7"></a>
### <b> <font color='#5499C7'> 4.7. Prepare datasets for TensorFlow </font> </b>

In [37]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_vectors, 
                                              y_train.values))


validation_dataset = tf.data.Dataset.from_tensor_slices((X_val_vectors, 
                                              y_val.values))


test_dataset = tf.data.Dataset.from_tensor_slices((X_test_vectors, 
                                              y_test.values))


In [38]:
# let's see an example for train
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  [ 822  744  232   16 3337   45  802    1 1261    1  294   35   20 2686
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]
label:  [1. 0. 0.]


We define the size of the buffer and the size of the batch:

In [39]:
BUFFER_SIZE = 10000
BATCH_SIZE = 32

In [40]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

Let's see some examples and their labels

In [41]:
for example, label in train_dataset.take(1):
  print(f'texts:  {example.numpy()[:3]}\n')
  print(f'labels: , {label.numpy()[:3]}')

texts:  [[ 553  159  251 1925 1149 2191  117  357   25    6   61    7   34  121
    53 2235  391 1774  131    9 2017  207 2082 1359 1898    1  946  125
  1807    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   1 1858   32   93 2206   42 3254   11  562  177    3 1534  134 3081
   898 5438    1  562   44   20  777   32    1    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [ 557    1 4661  370  185  115   62 4661  107    1 7153 1465  339  415
    23 1784  318 7499  324    8    5    2   46 1287    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]]

labels: , [[1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]


<a name="5"></a>
## <b> <font color='blue'> 5. Models </font> </b>

Let's try different models; all of them are going to have an embedding layer, that's why we are going to start by creating one of them.

<a name="5.1"></a>
### <b> <font color='#5499C7'> 5.1. Embedding layer </font> </b>


In [42]:
embedding = tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, # set the input shape; size of our vocabulary
                                 output_dim=128, # set the size of the embedding vector
                                 embeddings_initializer="uniform", # default, initialize embedding vectors randomly
                                 input_length=max_length # how long is each input
                             )

embedding

<a name="5.2"></a>
### <b> <font color='#5499C7'> 5.2. Model 1 </font> </b>

In [43]:
from tensorflow.keras import layers,callbacks,models,Sequential,losses

INPUT_SHAPE=(1,)

def build_model_1(input_shape,name):
    inputs = layers.Input(shape=input_shape)#, dtype=tf.string) # inputs are 1-dimensional strings
    #x = vectorizer(inputs) # turn the input text into numbers
    x = embedding(inputs)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    outputs = tf.keras.layers.Dense(3, activation="softmax")(x) # dsp.  num_classes
    model = tf.keras.Model(inputs, outputs, name=name) # construct the model
    return model


model_1 = build_model_1(INPUT_SHAPE,'model_1')


# Compile model
model_1.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy', 'Precision','Recall'])

# Fit the model
history_1 = model_1.fit(train_dataset,
                        epochs=5,
                        verbose=1,
                        validation_data=validation_dataset)

Epoch 1/5
1029/1029 [==============================] - 17s 16ms/step - loss: 0.8708 - accuracy: 0.6138 - precision: 0.8044 - recall: 0.3036 - val_loss: 0.7057 - val_accuracy: 0.7230 - val_precision: 0.8049 - val_recall: 0.5820
Epoch 2/5
1029/1029 [==============================] - 17s 16ms/step - loss: 0.5812 - accuracy: 0.7906 - precision: 0.8552 - recall: 0.6865 - val_loss: 0.5935 - val_accuracy: 0.7861 - val_precision: 0.8343 - val_recall: 0.7136
Epoch 3/5
1029/1029 [==============================] - 17s 17ms/step - loss: 0.4555 - accuracy: 0.8518 - precision: 0.8860 - recall: 0.7998 - val_loss: 0.5669 - val_accuracy: 0.8034 - val_precision: 0.8332 - val_recall: 0.7619
Epoch 4/5
1029/1029 [==============================] - 16s 16ms/step - loss: 0.3870 - accuracy: 0.8787 - precision: 0.9022 - recall: 0.8481 - val_loss: 0.5777 - val_accuracy: 0.8031 - val_precision: 0.8267 - val_recall: 0.7749
Epoch 5/5
1029/1029 [==============================] - 16s 16ms/step - loss: 0.3423 - accura

In [47]:
# evaluate
score1 = model_1.evaluate(test_dataset)

119/119 [==============================] - 0s 1ms/step - loss: 0.6224 - accuracy: 0.7844 - precision: 0.8031 - recall: 0.7562


In [52]:
def build_model_2(input_shape):
    inputs = layers.Input(shape=input_shape, dtype="float32")
    #x = encoder(inputs) # text vectorizer
    x = embedding(inputs) #x
    print(f"After embedding: {x.shape}")
    x = layers.LSTM(64, activation="tanh")(x)
    print(f"After LSTM cell: {x.shape}")
    x = layers.Dropout(0.5)(x) 
    x = layers.Dense(64, activation="relu")(x) # optional dense layer to have on top of LSTM layer
    outputs = layers.Dense(3, activation="softmax")(x)
    model = tf.keras.Model(inputs, outputs, name="model_2_LSTM")
    return model
    

model_2 = build_model_2(INPUT_SHAPE)

model_2.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy', 'Precision','Recall'])

# Fit the model
history_2 = model_2.fit(train_dataset,
                        epochs=5,
                        verbose=1,
                        validation_data=validation_dataset)

After embedding: (None, 1, 128)
After LSTM cell: (None, 64)
Epoch 1/5
1029/1029 [==============================] - 30s 27ms/step - loss: 0.3113 - accuracy: 0.8961 - precision: 0.9263 - recall: 0.8686 - val_loss: 0.7012 - val_accuracy: 0.7958 - val_precision: 0.8084 - val_recall: 0.7838
Epoch 2/5
1029/1029 [==============================] - 28s 27ms/step - loss: 0.2065 - accuracy: 0.9421 - precision: 0.9513 - recall: 0.9322 - val_loss: 0.7786 - val_accuracy: 0.7977 - val_precision: 0.8048 - val_recall: 0.7907
Epoch 3/5
1029/1029 [==============================] - 29s 28ms/step - loss: 0.1735 - accuracy: 0.9518 - precision: 0.9598 - recall: 0.9438 - val_loss: 0.8525 - val_accuracy: 0.7970 - val_precision: 0.8034 - val_recall: 0.7907
Epoch 4/5
1029/1029 [==============================] - 27s 26ms/step - loss: 0.1547 - accuracy: 0.9548 - precision: 0.9639 - recall: 0.9479 - val_loss: 0.8051 - val_accuracy: 0.7880 - val_precision: 0.8008 - val_recall: 0.7747
Epoch 5/5
1029/1029 [===========

In [53]:
# evaluate
score1 = model_2.evaluate(test_dataset)

119/119 [==============================] - 1s 5ms/step - loss: 0.8608 - accuracy: 0.7601 - precision: 0.7765 - recall: 0.7420
